In [1]:
from flask import Flask, request, jsonify
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import threading

app = Flask(__name__)

# Load the model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Load and process the new data
with open("cleaned_data.json", "r") as f:
    data = json.load(f)["questions"]

# Extract texts and encode them
docs = [{"text": q["question"] + " " + q["answer"]} for q in data]
corpus_embeddings = model.encode([d["text"] for d in docs])

# Create or load the FAISS index
dimension = corpus_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(corpus_embeddings))

def search_faiss(query, k=1):
    embedding = model.encode([query])
    D, I = index.search(np.array(embedding), k)
    results = [docs[i]["text"] for i in I[0]]
    return results[0] if results else "Sorry, I couldn't find anything relevant."

@app.route('/chat', methods=['POST'])
def chat():
    data = request.get_json()
    message = data.get("message", "")
    if not message:
        return jsonify({"response": "Please provide a message."})
    
    response = search_faiss(message)
    return jsonify({"response": response})

# Run server
def run_flask():
    app.run(host='0.0.0.0', port=5000)

if __name__ == "__main__":
    run_flask()


UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 19835: character maps to <undefined>